# 221804 商业分析基础｜评估任务一（Python 部分）

作者：<填写你的姓名>  
日期：2025-11-04

---
## 使用说明
1. 将 `leads.csv` 与 `new_leads.csv` 放在与本 Notebook 同一目录。
2. 依序运行每个单元。输出（表格/图/数值）可直接用于最终报告。
3. 若首次运行环境缺少依赖，请先运行 **0. 环境准备**。

## 0. 环境准备（可选）
- 如果本地没有安装依赖，先执行此单元。
- **注意**：Colab 环境请将 `--quiet` 去掉以便查看安装日志。

In [ ]:
import sys, subprocess
def pip_install(pkg):
    try:
        __import__(pkg.split('==')[0].replace('-','_'))
        print(f"✓ {pkg} 已安装")
    except Exception:
        print(f"→ 安装 {pkg} ...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg])

for p in [
    'pandas',
    'matplotlib',
    'seaborn',
    'pycaret==3.3.2',
    'scikit-learn'
]:
    pip_install(p)
print('环境准备完成。')

## 1. 导入库与全局设置

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.max_columns', 120)

from pathlib import Path
DATA_DIR = Path('.')  # 如需指定绝对路径，可修改此处
LEADS_PATH = DATA_DIR / 'leads.csv'
NEW_LEADS_PATH = DATA_DIR / 'new_leads.csv'
print('工作目录：', Path.cwd())
print('leads.csv 存在：', LEADS_PATH.exists())
print('new_leads.csv 存在：', NEW_LEADS_PATH.exists())

## 2. 读取数据并快速体检（Data Audit）
- 检查形状、缺失值、数据类型
- **与报告关联**：可直接摘录 `shape / info / describe` 里的关键数字

In [ ]:
df = pd.read_csv(LEADS_PATH)
print('数据维度:', df.shape)
display(df.head(5))
print('\n数据类型与非空计数：')
df.info()
display(df.describe(include='all').T)

# 常用列清单（可按需修改增删）
num_cols = ['TotalVisits', 'Total Time Spent on Website', 'Page Views Per Visit',
            'Activity Score', 'Profile Score']
cat_cols = ['Lead Origin', 'Lead Source', 'Last Notable Activity']

# 缺失率
miss_rate = df.isna().mean().sort_values(ascending=False)
display(miss_rate.to_frame('missing_rate').head(10))

## 3. 核心问题 1：潜在客户来自哪里？
- 与课堂任务对应：`Lead Origin` / `Lead Source` 的频数统计
- **与报告关联**：可放一张柱状图 + 简短文字

In [ ]:
origin_counts = df['Lead Origin'].value_counts(dropna=False)
source_counts = df['Lead Source'].value_counts(dropna=False)
display(origin_counts)
display(source_counts)

plt.figure(figsize=(10,4))
origin_counts.plot(kind='bar')
plt.title('Lead Origin 频数')
plt.xlabel('Lead Origin')
plt.ylabel('Count')
plt.tight_layout(); plt.show()

plt.figure(figsize=(10,4))
source_counts.head(15).plot(kind='bar')
plt.title('Lead Source 频数（Top 15）')
plt.xlabel('Lead Source')
plt.ylabel('Count')
plt.tight_layout(); plt.show()

## 4. 核心问题 2：行为变量的典型值与分布
- 与课堂任务对应：`Total Time Spent on Website`、`Page Views Per Visit` 的统计与直方图
- **与报告关联**："典型"＝中位数/四分位；指出右偏/极端值并控制坐标范围

In [ ]:
behav_cols = ['Total Time Spent on Website', 'TotalVisits', 'Page Views Per Visit']
display(df[behav_cols].describe().T)

fig, axes = plt.subplots(1,3, figsize=(15,4))
for ax, col in zip(axes, behav_cols):
    ax.hist(df[col].dropna(), bins=30)
    ax.set_title(col)
plt.tight_layout(); plt.show()

# 控制坐标：TotalVisits（0-15）
plt.figure(figsize=(6,4))
plt.hist(df['TotalVisits'].dropna(), bins=np.arange(-0.5, 15.5, 1))
plt.xlim(0,15)
plt.title('TotalVisits（限制 0–15）')
plt.xlabel('Visits'); plt.ylabel('Count')
plt.tight_layout(); plt.show()

## 5. 行为变量与转化的关系（示例）
- 与课堂任务对应：变量 vs `Converted` 的关系
- **与报告关联**：挑选一个变量做更深入解读（图 + 统计 + 口径说明）

In [ ]:
# 以 Total Time Spent on Website 为例，比较不同转化状态的分布
col = 'Total Time Spent on Website'
plt.figure(figsize=(8,4))
for label, grp in df.groupby('Converted'):
    plt.hist(grp[col].dropna(), bins=30, alpha=0.4, label=f'Converted={label}')
plt.legend(); plt.title(f'{col} vs Converted'); plt.tight_layout(); plt.show()

# 分组统计（均值/中位数）
group_stats = df.groupby('Converted')[behav_cols].agg(['mean','median'])
display(group_stats)

## 6. 预测建模（PyCaret）
- 目标：`Converted`（二分类）
- 步骤：`setup` → `compare_models` → 解读特征重要性与混淆矩阵
- **注意**：首次运行时间取决于机器性能。

In [ ]:
from pycaret.classification import setup, compare_models, plot_model, predict_model, get_config, add_metric, pull

s = setup(data=df, target='Converted', session_id=42, silent=True, verbose=False)
best_model = compare_models()
best_model

In [ ]:
# 特征重要性
plot_model(best_model, plot='feature')
# 混淆矩阵
plot_model(best_model, plot='confusion_matrix')

## 7. 指标计算（Accuracy / FPR / FNR）
- 从 Hold-out 预测重建混淆矩阵并计算指标
- **与报告关联**：可直接引用数值并解释假阳/假阴的业务含义

In [ ]:
from sklearn.metrics import confusion_matrix

hold = predict_model(best_model)
y_true = get_config('y_test')
y_pred = get_config('pred_holdout')
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()
ACC = (TP + TN) / (TP + TN + FP + FN)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
print(f'Accuracy: {ACC:.2%}\nFPR: {FPR:.2%}\nFNR: {FNR:.2%}')

## 8. 业务指标：利润（可切换情景）
- 情景 A：每次跟进成本 $15；成功转化增量收入 $120
- 情景 B：每次跟进成本 $20；其余不变
- **与报告关联**：对比不同成本下 Profit 的变化

In [ ]:
def calculate_profit(actual, predicted, sales_expense=15, revenue=120):
    tp = np.where((predicted==1) & (actual==1), (revenue - sales_expense), 0)
    fp = np.where((predicted==1) & (actual==0), (-sales_expense), 0)
    return np.sum([tp, fp])

# 情景 A（cost=15）
add_metric('profit', 'Profit', lambda y, p: calculate_profit(y, p, 15, 120), greater_is_better=True)
preds_train = predict_model(best_model)
train_profit_15 = pull().get('Profit', None)
print('Train Profit (cost=15):', train_profit_15)

# 情景 B（cost=20）
add_metric('profit_20', 'Profit_20', lambda y, p: calculate_profit(y, p, 20, 120), greater_is_better=True)
preds_train_20 = predict_model(best_model)
train_profit_20 = pull().get('Profit_20', None)
print('Train Profit (cost=20):', train_profit_20)

## 9. 新潜在客户预测与基线对比
- 使用最佳模型对 `new_leads.csv` 进行预测
- 与“联系所有人”的基线策略比较利润差异

In [ ]:
new_leads = pd.read_csv(NEW_LEADS_PATH)
new_preds = predict_model(best_model, data=new_leads, raw_score=True)
eval_new = pull()  # 含 Accuracy/AUC/Profit 等
display(eval_new)

profit_model = eval_new.get('Profit', None) or eval_new.get('Profit_20', None)

# 基线：联系所有人（与情景 A 对齐：cost=15，收入=120）
cost, revenue = 15, 120
n_total = new_preds['Converted'].count()
n_conv = new_preds['Converted'].sum()
profit_all = n_conv * revenue - n_total * cost

print(f"New leads: {n_total} 条；预测转化：{n_conv}")
print('Profit (model):', profit_model)
print('Profit (contact everyone):', profit_all)
print('Δ Profit = model - everyone:', None if profit_model is None else profit_model - profit_all)

## 10. 结果小结（供报告引用）
- 用 3–5 条要点总结：
  1. 关键来源/渠道 & 其转化差异；
  2. 1–2 个**强相关**行为变量的“典型值区间”和与转化的关系；
  3. 最佳模型与核心指标（AUC/Accuracy/FPR/FNR）；
  4. 利润情景（cost=15 vs 20）对结论的影响；
  5. 新潜在客户的利润对比与建议（是否采用模型、阈值/优先级）。

---
### 附录：复现与可移植性
- 固定随机种子 `session_id=42` 以减少波动；
- 若 `plot_model` 在远程环境不显示，请在本地运行或保留表格截图；
- 运行顺序：自上而下，无需手动跳步。